## Capstone project

## Read in the new sample file with only 10 HCPCS code

In [2]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
plt.style.use('ggplot')
%matplotlib inline

In [3]:
hcpcs10 = pd.read_csv('top10_hcpcs.csv', index_col=0)

# hcpcs10 = pd.read_csv('top10_hcpcs.csv', index_col=0)
# hcpcs10 = hcpcs10.reset_index()

C:\Users\aparn\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\aparn\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
pd.set_option("display.max_columns",500)
hcpcs10.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt
6,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00
7,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99232,"Subsequent hospital inpatient care, typically ...",N,627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94
8,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99233,"Subsequent hospital inpatient care, typically ...",N,207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88
10,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99291,Critical care delivery critically ill or injur...,N,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00
39,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,99213,Established patient office or other outpatient...,N,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98


In [5]:
hcpcs10.dtypes

npi                                   int64
nppes_provider_last_org_name         object
nppes_provider_first_name            object
nppes_provider_mi                    object
nppes_credentials                    object
nppes_provider_gender                object
nppes_entity_code                    object
nppes_provider_street1               object
nppes_provider_street2               object
nppes_provider_city                  object
nppes_provider_zip                   object
nppes_provider_state                 object
nppes_provider_country               object
provider_type                        object
medicare_participation_indicator     object
place_of_service                     object
hcpcs_code                           object
hcpcs_description                    object
hcpcs_drug_indicator                 object
line_srvc_cnt                       float64
bene_unique_cnt                       int64
bene_day_srvc_cnt                     int64
average_Medicare_allowed_amt    

In [9]:
hcpcs10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520985 entries, 6 to 9847441
Data columns (total 28 columns):
npi                                 1520985 non-null int64
nppes_provider_last_org_name        1520959 non-null object
nppes_provider_first_name           1511915 non-null object
nppes_provider_mi                   1075160 non-null object
nppes_credentials                   1455318 non-null object
nppes_provider_gender               1511939 non-null object
nppes_entity_code                   1520985 non-null object
nppes_provider_street1              1520983 non-null object
nppes_provider_street2              632192 non-null object
nppes_provider_city                 1520985 non-null object
nppes_provider_zip                  1520985 non-null object
nppes_provider_state                1520985 non-null object
nppes_provider_country              1520985 non-null object
provider_type                       1520985 non-null object
medicare_participation_indicator    1520985 non-n

In [6]:
hcpcs10.columns[hcpcs10.isnull().any()].tolist()

['nppes_provider_last_org_name',
 'nppes_provider_first_name',
 'nppes_provider_mi',
 'nppes_credentials',
 'nppes_provider_gender',
 'nppes_provider_street1',
 'nppes_provider_street2']

In [7]:
## Get a count of the missings by column.

len(hcpcs10.index) - hcpcs10.count()

npi                                      0
nppes_provider_last_org_name            26
nppes_provider_first_name             9070
nppes_provider_mi                   445825
nppes_credentials                    65667
nppes_provider_gender                 9046
nppes_entity_code                        0
nppes_provider_street1                   2
nppes_provider_street2              888793
nppes_provider_city                      0
nppes_provider_zip                       0
nppes_provider_state                     0
nppes_provider_country                   0
provider_type                            0
medicare_participation_indicator         0
place_of_service                         0
hcpcs_code                               0
hcpcs_description                        0
hcpcs_drug_indicator                     0
line_srvc_cnt                            0
bene_unique_cnt                          0
bene_day_srvc_cnt                        0
average_Medicare_allowed_amt             0
average_sub

In [49]:
hcpcs10.loc[:,['hcpcs_code','hcpcs_description','line_srvc_cnt','average_submitted_chrg_amt','average_Medicare_payment_amt']]

,hcpcs_code,hcpcs_description,line_srvc_cnt,average_submitted_chrg_amt,average_Medicare_payment_amt
6,99223,"Initial hospital inpatient care, typically 70 ...",114.0,923.991228,160.377193
7,99232,"Subsequent hospital inpatient care, typically ...",627.0,326.245614,56.366730
8,99233,"Subsequent hospital inpatient care, typically ...",207.0,461.072464,82.820676
10,99291,Critical care delivery critically ill or injur...,36.0,1400.000000,175.750000
39,99213,Established patient office or other outpatient...,129.0,109.000000,50.868062
40,99214,Established patient office or other outpatient...,24.0,115.000000,58.658333
41,99214,Established patient office or other outpatient...,330.0,159.000000,73.218515
48,99223,"Initial hospital inpatient care, typically 70 ...",74.0,243.391892,149.435000
51,99232,"Subsequent hospital inpatient care, typically ...",241.0,150.000000,53.631328
52,99233,"Subsequent hospital inpatient care, typically ...",76.0,175.000000,78.250000


In [30]:
# to get the fifrst 5 digits of zip code, first convert the zip code var to string. 

hcpcs10["nppes_provider_zip"]= hcpcs10["nppes_provider_zip"].astype(str) 

In [31]:
# Now use string slicing to get the first five digits.

hcpcs10['prov_zip'] = hcpcs10.nppes_provider_zip.str.slice(0,5,1)

In [32]:
hcpcs10.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,prov_zip
6,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,21502
7,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99232,"Subsequent hospital inpatient care, typically ...",N,627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,21502
8,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99233,"Subsequent hospital inpatient care, typically ...",N,207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,21502
10,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99291,Critical care delivery critically ill or injur...,N,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,21502
39,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,99213,Established patient office or other outpatient...,N,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,43623


In [37]:
leie_m = pd.read_csv('LEIE_ALL_17_19.csv', dtype={'ZIP': object})

#leie_m = pd.read_csv('LEIE_ALL_17_19', index_col=0)

In [38]:
leie_m.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,EXCLYEAR
0,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,2018-04-19,0,0,NaN,2018
1,NaN,NaN,,"716 TRANSPORTATION, INC",OTHER BUSINESS,TRANSPORTATION CO,NaN,0,NaN,540 VINE LANE,BUFFALO,NY,14228,1128a1,2019-06-20,0,0,NaN,2019
2,NaN,NaN,,A & H VITAMINS SUPPLY CORP,OTHER BUSINESS,PHARMACY,NaN,0,NaN,592 PALISADE AVENUE,TEANECK,NJ,7666,1128a1,2018-04-19,0,0,NaN,2018
3,NaN,NaN,,"A & Y MEDICAL SUPPLY, INC",DME COMPANY,DME - GENERAL,NaN,1942476080,NaN,"6310 108TH STREET, APT 6J",FOREST HILLS,NY,11375,1128b8,2017-05-18,0,0,NaN,2017
4,NaN,NaN,,"A FAIR DEAL PHARMACY, INC",OTHER BUSINESS,PHARMACY,NaN,1891731758,NaN,"C/O P O BOX 329014, #69709-05",BROOKLYN,NY,11232,1128b8,2017-05-18,0,0,NaN,2017


In [39]:
leie_m.dtypes

LASTNAME       object
FIRSTNAME      object
MIDNAME        object
BUSNAME        object
GENERAL        object
SPECIALTY      object
UPIN           object
NPI             int64
DOB           float64
ADDRESS        object
CITY           object
STATE          object
ZIP            object
EXCLTYPE       object
EXCLDATE       object
REINDATE        int64
WAIVERDATE      int64
WVRSTATE      float64
EXCLYEAR        int64
dtype: object

In [41]:
## Check the length of the zip code var. Create a variable that stores the length of each zip code and freq it.

leie_m['zip_len']=leie_m['ZIP'].str.len()

In [50]:
# Obtain the frequency counts

leie_m['zip_len'].value_counts()

5    7120
4     540
3      22
1       5
Name: zip_len, dtype: int64

In [44]:
## Since there are several that are not 5 digit codes, add leading zeros to them. NJ codes have leading zeros.
## This works for the 4 digit codes
## There are 27 that are 3 or 1 digit long. These are problems that we can drop if necessary.

leie_m['zip_new']= leie_m['ZIP'].apply(lambda x: x.zfill(5))

In [45]:
leie_m.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,EXCLYEAR,zip_len,zip_new
0,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,2018-04-19,0,0,NaN,2018,5,10468
1,NaN,NaN,,"716 TRANSPORTATION, INC",OTHER BUSINESS,TRANSPORTATION CO,NaN,0,NaN,540 VINE LANE,BUFFALO,NY,14228,1128a1,2019-06-20,0,0,NaN,2019,5,14228
2,NaN,NaN,,A & H VITAMINS SUPPLY CORP,OTHER BUSINESS,PHARMACY,NaN,0,NaN,592 PALISADE AVENUE,TEANECK,NJ,7666,1128a1,2018-04-19,0,0,NaN,2018,4,07666
3,NaN,NaN,,"A & Y MEDICAL SUPPLY, INC",DME COMPANY,DME - GENERAL,NaN,1942476080,NaN,"6310 108TH STREET, APT 6J",FOREST HILLS,NY,11375,1128b8,2017-05-18,0,0,NaN,2017,5,11375
4,NaN,NaN,,"A FAIR DEAL PHARMACY, INC",OTHER BUSINESS,PHARMACY,NaN,1891731758,NaN,"C/O P O BOX 329014, #69709-05",BROOKLYN,NY,11232,1128b8,2017-05-18,0,0,NaN,2017,5,11232


In [48]:
leie_m.shape

(7687, 21)